<a href="https://colab.research.google.com/github/RaphaelCarvalh/clothing-segmentation-project/blob/main/02_training_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 02 Pipeline de Treinamento

#Montando o baseline

Baixando dataset direto do kaggle

In [1]:
# ================================
# Setup Kaggle Dataset
# ================================
!pip install opendatasets opencv-python-headless pandas pillow matplotlib seaborn torch torchvision --quiet

import os
import opendatasets as od
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

import os
import opendatasets as od

BASE_DIR = "/content/clothing-coparsing-dataset"
DATASET_URL = "https://www.kaggle.com/datasets/balraj98/clothing-coparsing-dataset"

if not os.path.exists(BASE_DIR):
    print("Dataset não encontrado. Fazendo download do Kaggle...")
    print("IMPORTANTE: insira suas credenciais da API do Kaggle (kaggle.json).")
    od.download(DATASET_URL, data_dir="/content/")
    print(f"Dataset baixado em: {BASE_DIR}")
else:
    print(f"Dataset já disponível em: {BASE_DIR}")


Dataset já disponível em: /content/clothing-coparsing-dataset


Se o repo é público, você pode usar o link bruto do GitHub:

In [2]:
!pip install pyarrow fastparquet --quiet

In [3]:
import pandas as pd

PARQUET_URL = "https://github.com/RaphaelCarvalh/clothing-segmentation-project/raw/main/data/df_valido.parquet"

df_valido = pd.read_parquet(PARQUET_URL, engine="pyarrow")
print(f"Total válido carregado: {len(df_valido)} linhas")
df_valido.head()

Total válido carregado: 1003 linhas


,image_path_full,image_path_relative,label_path_full,label_path_relative,image_exists,label_exists,corrupted,image_hash,image_format,width,height,channels,is_duplicate,classes,num_classes,image_id
0,/content/clothing-coparsing-dataset/images/000...,images/0001.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0001.png,True,True,False,9349b423c4d40aea33b9334be15050ef,jpeg,550,832,3,False,"[hair, shoes, skin, blouse, skirt, stockings, ...",8,1
1,/content/clothing-coparsing-dataset/images/000...,images/0002.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0002.png,True,True,False,7d2824d0e04ca4fa0575fb88173a7cc1,jpeg,550,830,3,False,"[pumps, jacket, hair, dress, skin, socks]",6,2
2,/content/clothing-coparsing-dataset/images/000...,images/0003.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0003.png,True,True,False,b3671739ed055b6b2cb21bd12fab7627,jpeg,550,831,3,False,"[pants, purse, necklace, jacket, hair, dress, ...",13,3
3,/content/clothing-coparsing-dataset/images/000...,images/0004.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0004.png,True,True,False,886cfd5b1269d5c7d2566a202395ce4d,jpeg,550,810,3,False,"[pants, hat, shirt, glasses, shoes, coat, skin...",9,4
4,/content/clothing-coparsing-dataset/images/000...,images/0005.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0005.png,True,True,False,cb5c8591a2e29740db0601be07d80cc4,jpeg,550,842,3,False,"[purse, hair, shoes, dress, cape, skin, sungla...",8,5


Puxando o df com dados do dataset ja tratado Obs. Versão mais completa do import caso o repo fique privado em algum momento.

In [4]:
# import getpass
# import pandas as pd
# import requests
# from io import StringIO

# # Pede token de forma segura
# GITHUB_TOKEN = getpass.getpass("Digite seu token GitHub: ")

# # URL do arquivo CSV
# CSV_PATH = "data/df_valido.csv"
# REPO = "RaphaelCarvalh/clothing-segmentation-project"
# RAW_URL = f"https://raw.githubusercontent.com/{REPO}/main/{CSV_PATH}"

# # Autenticação com token
# headers = {"Authorization": f"token {GITHUB_TOKEN}"}
# response = requests.get(RAW_URL, headers=headers)

# # Ler CSV na memória
# df_valido = pd.read_csv(StringIO(response.text))
# print(f"Total válido carregado: {len(df_valido)} linhas")
# df_valido.head()


In [4]:
import os

# Caminhos base no Colab após download do Kaggle
IMAGES_DIR = os.path.join(BASE_DIR, "images")
LABELS_DIR = os.path.join(BASE_DIR, "labels/pixel_level_labels_colored")

print("Estrutura base confirmada:")
print(f"IMAGES_DIR: {IMAGES_DIR}")
print(f"LABELS_DIR: {LABELS_DIR}")

# Função para corrigir paths se necessário
def ajustar_paths(df):
    df_corrigido = df.copy()

    if "image_path_full" in df_corrigido.columns:
        df_corrigido["image_path_full"] = df_corrigido["image_path_full"].apply(
            lambda x: os.path.join(IMAGES_DIR, os.path.basename(x)) if isinstance(x, str) else x
        )

    if "label_path_full" in df_corrigido.columns:
        df_corrigido["label_path_full"] = df_corrigido["label_path_full"].apply(
            lambda x: os.path.join(LABELS_DIR, os.path.basename(x)) if isinstance(x, str) else x
        )

    return df_corrigido

# Corrige e valida
df_valido = ajustar_paths(df_valido)

# Checar integridade
print("\nValidação dos caminhos após ajuste:")
print(f"Imagens encontradas: {df_valido['image_path_full'].apply(os.path.exists).sum()} / {len(df_valido)}")
print(f"Labels encontradas: {df_valido['label_path_full'].apply(os.path.exists).sum()} / {len(df_valido)}")

df_valido.head()

Estrutura base confirmada:
IMAGES_DIR: /content/clothing-coparsing-dataset/images
LABELS_DIR: /content/clothing-coparsing-dataset/labels/pixel_level_labels_colored

Validação dos caminhos após ajuste:
Imagens encontradas: 1003 / 1003
Labels encontradas: 1003 / 1003


,image_path_full,image_path_relative,label_path_full,label_path_relative,image_exists,label_exists,corrupted,image_hash,image_format,width,height,channels,is_duplicate,classes,num_classes,image_id
0,/content/clothing-coparsing-dataset/images/000...,images/0001.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0001.png,True,True,False,9349b423c4d40aea33b9334be15050ef,jpeg,550,832,3,False,"[hair, shoes, skin, blouse, skirt, stockings, ...",8,1
1,/content/clothing-coparsing-dataset/images/000...,images/0002.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0002.png,True,True,False,7d2824d0e04ca4fa0575fb88173a7cc1,jpeg,550,830,3,False,"[pumps, jacket, hair, dress, skin, socks]",6,2
2,/content/clothing-coparsing-dataset/images/000...,images/0003.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0003.png,True,True,False,b3671739ed055b6b2cb21bd12fab7627,jpeg,550,831,3,False,"[pants, purse, necklace, jacket, hair, dress, ...",13,3
3,/content/clothing-coparsing-dataset/images/000...,images/0004.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0004.png,True,True,False,886cfd5b1269d5c7d2566a202395ce4d,jpeg,550,810,3,False,"[pants, hat, shirt, glasses, shoes, coat, skin...",9,4
4,/content/clothing-coparsing-dataset/images/000...,images/0005.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0005.png,True,True,False,cb5c8591a2e29740db0601be07d80cc4,jpeg,550,842,3,False,"[purse, hair, shoes, dress, cape, skin, sungla...",8,5


#Dividir o dataset em treino, validação e teste

In [5]:
from sklearn.model_selection import train_test_split

# Configurar proporções (ex: 70% train, 15% val, 15% test)
train_size = 0.7
val_size = 0.15
test_size = 0.15

# Primeiro: separar train e temp (val+test)
df_train, df_temp = train_test_split(
    df_valido,
    train_size=train_size,
    random_state=42,
    shuffle=True
)

# Depois: dividir temp em val e test
relative_val_size = val_size / (val_size + test_size)  # ajuste proporcional
df_val, df_test = train_test_split(
    df_temp,
    train_size=relative_val_size,
    random_state=42,
    shuffle=True
)

print("Tamanhos finais:")
print(f"Train: {len(df_train)} imagens")
print(f"Val:   {len(df_val)} imagens")
print(f"Test:  {len(df_test)} imagens")

# Confirmar que não houve sobreposição
assert set(df_train.index).isdisjoint(df_val.index)
assert set(df_train.index).isdisjoint(df_test.index)
assert set(df_val.index).isdisjoint(df_test.index)

# Conferir algumas amostras
print("\nExemplo Train:")
display(df_train.head())
print("\nExemplo Val:")
display(df_val.head())
print("\nExemplo Test:")
display(df_test.head())


Tamanhos finais:
Train: 702 imagens
Val:   150 imagens
Test:  151 imagens

Exemplo Train:


,image_path_full,image_path_relative,label_path_full,label_path_relative,image_exists,label_exists,corrupted,image_hash,image_format,width,height,channels,is_duplicate,classes,num_classes,image_id
155,/content/clothing-coparsing-dataset/images/015...,images/0157.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0157.png,True,True,False,71f0277dc6461b27bb81b03e32739c13,jpeg,550,843,3,False,"[purse, leggings, hair, shoes, skin, accessori...",9,157
542,/content/clothing-coparsing-dataset/images/054...,images/0544.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0544.png,True,True,False,1f289ae6d9a05b73036ad531d65811f6,jpeg,550,818,3,False,"[purse, hair, shoes, dress, skin]",5,544
442,/content/clothing-coparsing-dataset/images/044...,images/0444.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0444.png,True,True,False,0f2ab70d6be5213586fb23472a0a824d,jpeg,550,835,3,False,"[purse, shirt, hair, shoes, coat, bracelet, sk...",8,444
819,/content/clothing-coparsing-dataset/images/082...,images/0821.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0821.png,True,True,False,a5d2abbbbba7c0a0d754e4c3863790d7,jpeg,550,829,3,False,"[hair, shoes, shorts, bracelet, skin, t-shirt]",6,821
398,/content/clothing-coparsing-dataset/images/040...,images/0400.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0400.png,True,True,False,880c26901effc8c2f0a938443a11b112,jpeg,550,825,3,False,"[pants, hair, shoes, skin, bag, suit]",6,400



Exemplo Val:


,image_path_full,image_path_relative,label_path_full,label_path_relative,image_exists,label_exists,corrupted,image_hash,image_format,width,height,channels,is_duplicate,classes,num_classes,image_id
277,/content/clothing-coparsing-dataset/images/027...,images/0279.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0279.png,True,True,False,7997c5cb88c47a04b321925d41fdd32c,jpeg,550,849,3,False,"[purse, hair, shoes, skin, blouse, skirt]",6,279
576,/content/clothing-coparsing-dataset/images/057...,images/0578.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0578.png,True,True,False,634ebfbcd2c3307b567b74350ed1bb02,jpeg,550,816,3,False,"[leggings, hair, shoes, dress, coat, cardigan,...",7,578
886,/content/clothing-coparsing-dataset/images/088...,images/0888.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0888.png,True,True,False,aa64019a017ffb855e9c7efc836d3a19,jpeg,550,835,3,False,"[purse, hair, shoes, dress, skin, accessories,...",7,888
944,/content/clothing-coparsing-dataset/images/094...,images/0946.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0946.png,True,True,False,056cdfb4b2f2f15c4013da4ef737eb28,jpeg,550,836,3,False,"[purse, necklace, sandals, hair, dress, skin, ...",8,946
622,/content/clothing-coparsing-dataset/images/062...,images/0624.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0624.png,True,True,False,c189bbcb91600dab367e648a3d83f26d,jpeg,550,837,3,False,"[purse, hair, shoes, dress, bracelet, skin]",6,624



Exemplo Test:


,image_path_full,image_path_relative,label_path_full,label_path_relative,image_exists,label_exists,corrupted,image_hash,image_format,width,height,channels,is_duplicate,classes,num_classes,image_id
587,/content/clothing-coparsing-dataset/images/058...,images/0589.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0589.png,True,True,False,dd3586f81e7c086cd7d99217994b545a,jpeg,550,839,3,False,"[pants, scarf, hair, shoes, coat, skin, bag, s...",8,589
451,/content/clothing-coparsing-dataset/images/045...,images/0453.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0453.png,True,True,False,664b1be13edce0f4f9c160778746ef63,jpeg,550,827,3,False,"[pants, purse, shirt, hair, shoes, skin, acces...",9,453
545,/content/clothing-coparsing-dataset/images/054...,images/0547.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0547.png,True,True,False,b53a674b1ccf57cd7f5be282a08350fe,jpeg,550,823,3,False,"[pants, purse, hair, shoes, skin, sunglasses, ...",7,547
334,/content/clothing-coparsing-dataset/images/033...,images/0336.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0336.png,True,True,False,69c0724ecef6f8c5476527068f6f2a3f,jpeg,550,813,3,False,"[pants, hat, shirt, shoes, coat, skin, bag, be...",9,336
953,/content/clothing-coparsing-dataset/images/095...,images/0955.jpg,/content/clothing-coparsing-dataset/labels/pix...,labels/pixel_level_labels_colored/0955.png,True,True,False,57c8e590a0424d7c0f44b126b88770ec,jpeg,550,827,3,False,"[romper, hair, gloves, flats, coat, skin, bag]",7,955


# Dataset e DataLoader

In [6]:
import torch
from torch.utils.data import DataLoader

torch.cuda.is_available()

class ClothingSegmentationDataset(Dataset):
    def __init__(self, df, transform_img=None, transform_mask=None):
        self.df = df
        self.transform_img = transform_img
        self.transform_mask = transform_mask

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["image_path_full"]).convert("RGB")
        mask = Image.open(row["label_path_full"]).convert("L")

        if self.transform_img:
            img = self.transform_img(img)
        if self.transform_mask:
            mask = self.transform_mask(mask)

        return img, mask

# Transforms
train_img_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_mask_transforms = transforms.Compose([
    transforms.Resize((256, 256), interpolation=Image.NEAREST),
    transforms.ToTensor()
])

# DataLoaders
train_dataset = ClothingSegmentationDataset(df_train, train_img_transforms, train_mask_transforms)
val_dataset = ClothingSegmentationDataset(df_val, train_img_transforms, train_mask_transforms)
test_dataset = ClothingSegmentationDataset(df_test, train_img_transforms, train_mask_transforms)

BATCH_SIZE = 14
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=False)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=False)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=False)

# Configurar modelo DeepLabV3 (segmentação)

In [7]:
import torch
from torchvision.models.segmentation import deeplabv3_resnet50

device = "cpu"
print("Dispositivo:", device)

num_classes = 20  # ajuste para o número real de classes
model = deeplabv3_resnet50(weights="DEFAULT")
model.classifier[-1] = torch.nn.Conv2d(256, num_classes, kernel_size=(1,1))
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


Dispositivo: cpu


#Funções de treino e avaliação

In [8]:
from tqdm import tqdm

def train_epoch_progress(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for imgs, masks in tqdm(loader, desc="Treinando", leave=False):
        imgs = imgs.to(device)
        masks = masks.squeeze(1).long().to(device)
        optimizer.zero_grad()
        outputs = model(imgs)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def eval_epoch_progress(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for imgs, masks in tqdm(loader, desc="Validando", leave=False):
            imgs = imgs.to(device)
            masks = masks.squeeze(1).long().to(device)
            outputs = model(imgs)['out']
            loss = criterion(outputs, masks)
            total_loss += loss.item()
    return total_loss / len(loader)


#Loop de treinamento DeepLabV3 com ResNet-50

In [ ]:
from copy import deepcopy
import matplotlib.pyplot as plt
from IPython.display import clear_output

EPOCHS = 10
best_model = deepcopy(model)
best_loss = float("inf")

train_losses = []
val_losses = []

for epoch in range(1, EPOCHS+1):
    train_loss = train_epoch_progress(model, train_loader, criterion, optimizer, device)
    val_loss = eval_epoch_progress(model, val_loader, criterion, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    if val_loss < best_loss:
        best_model = deepcopy(model)
        best_loss = val_loss

    # Atualiza gráfico em tempo real
    clear_output(wait=True)
    plt.figure(figsize=(8,4))
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"Epoch {epoch}/{EPOCHS}")
    plt.legend()
    plt.show()

    print(f"Epoch {epoch}: Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f}")

Treinando:  14%|█▎        | 7/51 [09:20<57:08, 77.91s/it]

#Visualização de predições

In [ ]:
def mostrar_batch_predicoes(model, loader, device, n=4):
    model.eval()
    imgs, masks = next(iter(loader))
    imgs = imgs.to(device)
    with torch.no_grad():
        preds = model(imgs)['out'].argmax(1).cpu().numpy()

    plt.figure(figsize=(12, 6))
    for i in range(n):
        img = imgs[i].permute(1, 2, 0).cpu().numpy()
        img = (img * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)

        mask = masks[i].squeeze().cpu().numpy()
        pred_mask = preds[i]

        plt.subplot(3, n, i+1)
        plt.imshow(img)
        plt.title("Imagem")
        plt.axis("off")

        plt.subplot(3, n, i+1+n)
        plt.imshow(mask, cmap="tab20")
        plt.title("Máscara Real")
        plt.axis("off")

        plt.subplot(3, n, i+1+2*n)
        plt.imshow(pred_mask, cmap="tab20")
        plt.title("Máscara Predita")
        plt.axis("off")
    plt.tight_layout()
    plt.show()

# Exemplo
mostrar_batch_predicoes(best_model, test_loader, device, n=4)


# Segundo modelo para comparação

In [ ]:
!pip install segmentation-models-pytorch --quiet

In [ ]:

import segmentation_models_pytorch as smp

# Número de classes (igual ao dataset)
num_classes = 20

# U-Net com backbone ResNet-34
unet_model = smp.Unet(
    encoder_name="resnet34",        # backbone
    encoder_weights="imagenet",     # pesos pré-treinados
    in_channels=3,                  # imagens RGB
    classes=num_classes             # número de classes de saída
)

unet_model = unet_model.to(device)

criterion_unet = torch.nn.CrossEntropyLoss()
optimizer_unet = torch.optim.Adam(unet_model.parameters(), lr=1e-4)


#Loop de treinamento para U-Net básico com backbone ResNet-34 pré-treinado

In [ ]:
from copy import deepcopy

best_unet = deepcopy(unet_model)
best_loss_unet = float("inf")

for epoch in range(1, EPOCHS+1):
    train_loss = train_epoch(unet_model, train_loader, criterion_unet, optimizer_unet, device)
    val_loss = eval_epoch(unet_model, val_loader, criterion_unet, device)

    if val_loss < best_loss_unet:
        best_unet = deepcopy(unet_model)
        best_loss_unet = val_loss

    print(f"U-Net Epoch {epoch}: Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f}")

#Visualização das máscaras preditas pela U-Net

In [ ]:
mostrar_batch_predicoes(best_unet, test_loader, device, n=4)

#Funções para métricas

In [ ]:
import torch

def iou_score(preds, masks, num_classes):
    """
    Calcula IoU por classe e média.
    preds: tensor [B,H,W] com rótulos preditos
    masks: tensor [B,H,W] com rótulos verdadeiros
    """
    ious = []
    for cls in range(num_classes):
        pred_cls = (preds == cls)
        mask_cls = (masks == cls)
        intersection = (pred_cls & mask_cls).sum().item()
        union = (pred_cls | mask_cls).sum().item()
        iou = intersection / union if union > 0 else float('nan')
        ious.append(iou)
    mean_iou = np.nanmean(ious)
    return mean_iou, ious

def dice_score(preds, masks, num_classes):
    """
    Calcula Dice Score por classe e média.
    """
    dices = []
    for cls in range(num_classes):
        pred_cls = (preds == cls)
        mask_cls = (masks == cls)
        intersection = (pred_cls & mask_cls).sum().item()
        total = pred_cls.sum().item() + mask_cls.sum().item()
        dice = (2 * intersection) / total if total > 0 else float('nan')
        dices.append(dice)
    mean_dice = np.nanmean(dices)
    return mean_dice, dices


#Função de avaliação completa do modelo

In [ ]:
def evaluate_model(model, loader, device, num_classes):
    model.eval()
    all_iou = []
    all_dice = []
    with torch.no_grad():
        for imgs, masks in loader:
            imgs = imgs.to(device)
            masks = masks.squeeze(1).long().to(device)
            outputs = model(imgs)['out'] if hasattr(model, '__call__') else model(imgs)
            preds = outputs.argmax(1)

            iou, _ = iou_score(preds.cpu(), masks.cpu(), num_classes)
            dice, _ = dice_score(preds.cpu(), masks.cpu(), num_classes)

            all_iou.append(iou)
            all_dice.append(dice)

    mean_iou = np.mean(all_iou)
    mean_dice = np.mean(all_dice)
    return mean_iou, mean_dice


#Avaliar DeepLabV3

In [ ]:
iou_deeplab, dice_deeplab = evaluate_model(best_model, test_loader, device, num_classes)
print(f"DeepLabV3 - Mean IoU: {iou_deeplab:.4f}, Mean Dice: {dice_deeplab:.4f}")


#Avaliar U-Net

In [ ]:
iou_unet, dice_unet = evaluate_model(best_unet, test_loader, device, num_classes)
print(f"U-Net - Mean IoU: {iou_unet:.4f}, Mean Dice: {dice_unet:.4f}")


# Função para calcular métricas por classe

In [ ]:
def metrics_per_class(model, loader, device, num_classes):
    """
    Retorna IoU e Dice Score por classe.
    """
    model.eval()
    iou_all = np.zeros((len(loader), num_classes))
    dice_all = np.zeros((len(loader), num_classes))

    with torch.no_grad():
        for idx, (imgs, masks) in enumerate(loader):
            imgs = imgs.to(device)
            masks = masks.squeeze(1).long().to(device)
            outputs = model(imgs)['out'] if hasattr(model, '__call__') else model(imgs)
            preds = outputs.argmax(1)

            for cls in range(num_classes):
                pred_cls = (preds == cls)
                mask_cls = (masks == cls)
                intersection = (pred_cls & mask_cls).sum().item()
                union = (pred_cls | mask_cls).sum().item()
                total = pred_cls.sum().item() + mask_cls.sum().item()

                iou_all[idx, cls] = intersection / union if union > 0 else np.nan
                dice_all[idx, cls] = (2 * intersection) / total if total > 0 else np.nan

    mean_iou_per_class = np.nanmean(iou_all, axis=0)
    mean_dice_per_class = np.nanmean(dice_all, axis=0)

    return mean_iou_per_class, mean_dice_per_class


#Calcular métricas para ambos modelos

In [ ]:
iou_deeplab_cls, dice_deeplab_cls = metrics_per_class(best_model, test_loader, device, num_classes)
iou_unet_cls, dice_unet_cls = metrics_per_class(best_unet, test_loader, device, num_classes)


#Gráficos comparativos por classe

In [ ]:
classes = [f"Class {i}" for i in range(num_classes)]
x = np.arange(num_classes)
width = 0.35

fig, axes = plt.subplots(1, 2, figsize=(18,6))

# IoU
axes[0].bar(x - width/2, iou_deeplab_cls, width, label='DeepLabV3')
axes[0].bar(x + width/2, iou_unet_cls, width, label='U-Net')
axes[0].set_xticks(x)
axes[0].set_xticklabels(classes, rotation=45)
axes[0].set_ylim(0, 1)
axes[0].set_ylabel("IoU")
axes[0].set_title("IoU por Classe")
axes[0].legend()

# Dice
axes[1].bar(x - width/2, dice_deeplab_cls, width, label='DeepLabV3')
axes[1].bar(x + width/2, dice_unet_cls, width, label='U-Net')
axes[1].set_xticks(x)
axes[1].set_xticklabels(classes, rotation=45)
axes[1].set_ylim(0, 1)
axes[1].set_ylabel("Dice Score")
axes[1].set_title("Dice Score por Classe")
axes[1].legend()

plt.tight_layout()
plt.show()


#Calcular médias gerais

In [ ]:
# Média geral
mean_iou_deeplab = np.nanmean(iou_deeplab_cls)
mean_dice_deeplab = np.nanmean(dice_deeplab_cls)

mean_iou_unet = np.nanmean(iou_unet_cls)
mean_dice_unet = np.nanmean(dice_unet_cls)


# Preparar dados para plot

In [ ]:
metrics = ['IoU', 'Dice']
deeplab_vals = [mean_iou_deeplab, mean_dice_deeplab]
unet_vals = [mean_iou_unet, mean_dice_unet]

x = np.arange(len(metrics))
width = 0.35


#Plot comparativo resumido

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

ax.bar(x - width/2, deeplab_vals, width, label='DeepLabV3', color='skyblue')
ax.bar(x + width/2, unet_vals, width, label='U-Net', color='salmon')

ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.set_ylim(0, 1)
ax.set_ylabel("Média Geral")
ax.set_title("Comparação Geral de Métricas - DeepLabV3 vs U-Net")
ax.legend()

for i, v in enumerate(deeplab_vals):
    ax.text(i - width/2, v + 0.02, f"{v:.2f}", ha='center', fontweight='bold')
for i, v in enumerate(unet_vals):
    ax.text(i + width/2, v + 0.02, f"{v:.2f}", ha='center', fontweight='bold')

plt.tight_layout()
plt.show()


#Visualização em gráfico do comparitvo e tabela

In [ ]:
import pandas as pd

# Se tiver as losses de validação salvas
# val_loss_deeplab = val_loss[-1]  # última época do DeepLabV3
# val_loss_unet = val_loss_unet[-1]  # última época do U-Net

# Caso ainda não tenha, podemos colocar NaN ou calcular
val_loss_deeplab = np.nan
val_loss_unet = np.nan

data = {
    "Modelo": ["DeepLabV3 + ResNet50", "U-Net + ResNet34"],
    "Média IoU": [mean_iou_deeplab, mean_iou_unet],
    "Média Dice": [mean_dice_deeplab, mean_dice_unet],
    "Val Loss Final": [val_loss_deeplab, val_loss_unet]
}

df_comparativo = pd.DataFrame(data)
df_comparativo


fig, ax = plt.subplots(figsize=(10,6))
df_comparativo.set_index("Modelo")[["Média IoU", "Média Dice"]].plot(kind='bar', ax=ax, color=['skyblue', 'salmon'])
ax.set_ylim(0,1)
ax.set_ylabel("Valor Médio")
ax.set_title("Comparação Geral de Métricas entre Modelos")
plt.xticks(rotation=0)
plt.legend()
plt.tight_layout()
plt.show()
